In [8]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from tgym.core import Env, DataGenerator 
import pandas as pd
from sklearn.preprocessing import StandardScaler
import random

mpl.rcParams.update(
    {
        "font.size": 15,
        "axes.labelsize": 15,
        "lines.linewidth": 1,
        "lines.markersize": 8
    }
)
class SpreadTrading_simple(AbstractEnv):
     
    def step(self, action):
        self._params['action'] = action
        self._params['history_action'].append(self._params['action'])
        done = False
        Return = 0
#         print 'action :',self._params['action'],         
#         print 'price_history :',self._params['price_history'],
#         print 'observation0 :',self._params['observation'][0],
#         print 'observation1 :',self._params['observation'][1],
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        self._params['iteration'] += 1
        if self._params['iteration'] <= len(self._params['data']):
            self._params['history_data'] = self.gen_data( self._params['data'], self._params['iteration'] )
            self._params['price_history'] = self._params['history_data'][self._params['price_col']].tolist()
            Return_list = [( i - self._params['price_history'][-2] ) / self._params['price_history'][-2] for i in self._params['observation'][1]]
#             print(self._params['reward_gamma'],Return_list)
            weighted_return = self.reward_function(Return_list, self._params['reward_gamma'])
            if self._params['action'] == self._actions['buy']:
                if self._params['position'] < self._params['max_position'] : 
                    Return += weighted_return
                    Return += -1*self._params['trading_fee']
                    self._params['position'] += self._params['min_buy_position']
                    self._params['buy_price'] = self._params['price_history'][-2]                    
                else :
                    Return += -1*self._params['trading_fee']
                
            elif self._params['action'] == self._actions['hold']:
                if self._params['position'] == 0 :
                    Return = 0
                else:
                    Return += weighted_return
                    Return += -1*((1+self._params['time_fee'])**len(Return_list)-1)

            elif self._params['action'] == self._actions['sell']:
                if self._params['position'] == 0 : 
                    Return += -1*self._params['trading_fee']
                else :
                    Return += -1*weighted_return
                    Return += (1+self._params['time_fee'])**len(Return_list)-1
                    self._params['sell_price'] = self._params['price_history'][-2]
                    real_return = (self._params['sell_price'] - self._params['buy_price']) / self._params['buy_price'] - self._params['trading_fee']
                    self._params['accumulate_return'] = (real_return + 1) * (self._params['accumulate_return'] + 1) - 1 
                    self._params['position'] = 0

        self._params['done'], self._params['info'] = self.stop_condition()  
        if self._params['done'] :
            self._params['benchmark_sell_price'] = self._params['history_data'].tail(1)[self._params['price_col']].values[0]
            self._params['sell_price'] = self._params['price_history'][-1]
            self._params['benchmark_accumulate_return'] = \
            ( self._params['benchmark_sell_price'] - self._params['benchmark_buy_price'] ) / self._params['benchmark_buy_price'] - self._params['trading_fee']
            if (self._params['position'] > 0):
                real_return = (self._params['sell_price'] - self._params['buy_price']) / self._params['buy_price'] - self._params['trading_fee']
                self._params['accumulate_return'] = (real_return + 1) * (self._params['accumulate_return'] + 1) - 1 
#             print self._params['benchmark_buy_price'],self._params['benchmark_sell_price']
#             print 'benchmark_accumulate_return :',self._params['benchmark_accumulate_return']
        else:
            self._params['observation'] = self._get_observation()  
        self._params['accumulate_return_list'].append(self._params['accumulate_return'])
        self._params['reward_list'].append(Return*self._params['reward_scale'])
#         print 'Return :', Return, 
#         print 'done :', done
#         print 'accumulate_return :',self._params['accumulate_return']
#         print '========================================================'

        return self._params['observation'][0], Return*self._params['reward_scale'], self._params['done'], self._params['info']

NameError: name 'AbstractEnv' is not defined